# Live GPS Plotting from NMEA TCP Socket

This notebook connects to a local TCP socket streaming NMEA sentences from a GPS module, parses the data, and plots the live GPS track on an interactive map.

In [ ]:
from pynmeagps import NMEAMessage, haversine


def parse_nmea_latlon(msg: NMEAMessage):
    if msg.msgID == 'GGA' and  msg.lat != '':
        return (msg.lat, msg.lon)
    return None

def calc_distance_m(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    '''Calculates distance in meters using haversine formula'''
    return haversine(lat1, lon1, lat2, lon2) * 1000


In [ ]:
import socket
import time

from ipyleaflet import CircleMarker, LayerGroup, Map, Polyline, DivIcon, Marker
from pynmeagps import NMEAReader, NMEAMessage
from IPython.display import display

# Configuration
TCP_HOST = 'localhost'
TCP_PORT = 12345  # Change to your NMEA TCP port

# Map setup
INIT_CENTER = (51.0, 10.0)  # Default center (Germany)
m = Map(center=INIT_CENTER, zoom=14)
track_layer = LayerGroup()
m.add(track_layer)
m.layout.height = '600px'
display(m)

# Store all points for the polyline
track_points = []

# Store latest marker (as a Circle)
end_marker = None

poly = None
first_point = True

sock = None
speed_marker = None
speed = 0
last_pos = None
last_time = 0
try:
    sock = socket.create_connection((TCP_HOST, TCP_PORT))
    sock_file = sock.makefile('r', encoding='ascii')
    while len(line := sock_file.readline()) > 0:
        nmea_str = line.strip()
        nmea: NMEAMessage = NMEAReader.parse(nmea_str)
        if nmea.msgID == 'GGA' and nmea.quality == 1:
            pos = (nmea.lat, nmea.lon)
            track_points.append(pos)

            # Remove previous circle marker
            if end_marker:
                track_layer.remove(end_marker)
            if speed_marker:
                track_layer.remove(speed_marker)

            current_time = time.time()
            if last_pos is not None:
                dist = calc_distance_m(last_pos[0], last_pos[1], pos[0], pos[1])
                print(dist)
                speed = dist / (current_time - last_time) * 3.6
            print(speed)
            last_time = current_time
            last_pos = pos

            # Add a new circle marker for the latest point
            text_icon = DivIcon(
                html=f'<div style="font-size:12px; background:white; padding:2px 4px; border-radius:4px;">{speed:.1f} km/h</div>',
                icon_size=(100, 30),
                icon_anchor=(0, -15)  # adjust to position text above the circle
            )
            speed_marker = Marker(location=pos, icon=text_icon)
            end_marker = CircleMarker(location=pos, radius=4, color='blue', fill_color='blue', fill_opacity=0.8, weight=2)
            track_layer.add(end_marker)
            track_layer.add(speed_marker)

            # Remove previous polyline
            if poly:
                track_layer.remove(poly)

            # Draw polyline for the whole track
            poly = Polyline(locations=track_points, color='red', fill=False, weight=2)
            track_layer.add(poly)

            # Center map only on first datapoint
            if first_point:
                m.center = pos
                first_point = False

except KeyboardInterrupt:
    print('Stopped by user.')
finally:
    if sock:
        try:
            sock.shutdown(socket.SHUT_RDWR)
        except Exception:
            pass
        sock.close()
        print('Socket closed.')

In [ ]:
# Plot GPS track from a log file (NMEA sentences, one per line)
from datetime import datetime
from pathlib import Path
from typing import List, Tuple
import re
import os

from ipyleaflet import CircleMarker, LayerGroup, Map, Polyline, DivIcon, Marker
from IPython.display import display
from pynmeagps import NMEAMessage, NMEAReader

def parse_file_into_track(log_file: Path) -> List[Tuple[datetime, float, float]]:
    track_points = []
    with open(log_file, 'r', encoding='ascii') as f:
        for line in f:
            iso_str, nmea_str = line.strip().split(';')
            timestamp = datetime.fromisoformat(iso_str)
            nmea: NMEAMessage = NMEAReader.parse(nmea_str)
            if nmea.msgID == 'GGA' and nmea.quality == 1:
                pos = (timestamp, nmea.lat, nmea.lon)
                track_points.append(pos)
    return track_points

def draw_track(track_points: List[Tuple[datetime, float, float]]) -> LayerGroup:
    track_layer = LayerGroup()
    poly = Polyline(locations=[p[1:] for p in track_points], color='blue', fill=False, weight=2)
    track_layer.add(poly)
    # Start marker with timestamp
    start_pos = track_points[0][1:]
    start_time = track_points[0][0].strftime('%H:%M:%S')
    start_icon = DivIcon(html=f'<div style="font-size:12px; background:white; padding:1px 1px; ">{start_time}</div>', icon_size=(50, 20), icon_anchor=(0, -10))
    start_marker = CircleMarker(location=start_pos, radius=3, color='green', fill_color='green', fill_opacity=0.8, weight=2)
    start_time_marker = Marker(location=start_pos, icon=start_icon)
    track_layer.add(start_marker)
    track_layer.add(start_time_marker)
    # End marker with timestamp
    end_pos = track_points[-1][1:]
    end_time = track_points[-1][0].strftime('%H:%M:%S')
    end_icon = DivIcon(html=f'<div style="font-size:12px; background:white; padding:1px 1px; ">{end_time}</div>', icon_size=(50, 20), icon_anchor=(0, -10))
    end_marker = CircleMarker(location=end_pos, radius=3, color='red', fill_color='red', fill_opacity=0.8, weight=2)
    end_time_marker = Marker(location=end_pos, icon=end_icon)
    track_layer.add(end_marker)
    track_layer.add(end_time_marker)
    return track_layer


DIRECTORY = '/home/florian/Desktop/was-pod03/'
LOG_FILE_GLOB = '2025-07-28_*_gps.log'
LOG_FILES = sorted(Path(DIRECTORY).glob(LOG_FILE_GLOB))

tracks = []

for file in LOG_FILES:
    tracks.append(parse_file_into_track(file))

INIT_CENTER = tracks[0][0][1:]
m = Map(center=INIT_CENTER, zoom=14)
m.layout.height = '800px'

for track in tracks:
    m.add(draw_track(track))

m.save(LOG_FILE_GLOB.split('*')[0] + '.html')

display(m)